# Projet 7 : Implémentez un modèle de scoring




# Summary
 - <a href="#C1">Import and settings</a>
      - <a href="#C1.1">Import required items</a>
      - <a href="#C1.2">Settings</a>
 - <a href="#C2">Functions</a>
 - <a href="#C3">Data analysis and cleaning</a>


# <a name="C1"> Import and settings
## <a name="C1.1"> Import required items

In [1]:
# Use code formatting tool
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import required librairies
import os, time, datetime
import multiprocessing

import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, kernel_ridge, metrics
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
)

# from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from lightgbm import LGBMClassifier

# from sklearn import (
# cluster,
# metrics,
# manifold,
# decomposition,
# )

<IPython.core.display.Javascript object>

In [3]:
# Import data from file in a DataFrame
path = "Projet+Mise+en+prod+-+home-credit-default-risk/"

data_application_test = pd.read_csv(path + "application_test.csv")

data_application_train = pd.read_csv(path + "application_train.csv")

data_bureau = pd.read_csv(path + "bureau.csv")

data_bureau_balance = pd.read_csv(path + "bureau_balance.csv")

data_credit_card_balance = pd.read_csv(path + "credit_card_balance.csv")

data_col_desc = pd.read_csv(
    path + "HomeCredit_columns_description.csv", encoding="ISO-8859-1", index_col=0
)

data_installments_payments = pd.read_csv(path + "installments_payments.csv")

data_POS_CASH_balance = pd.read_csv(path + "POS_CASH_balance.csv")

data_previous_application = pd.read_csv(path + "previous_application.csv")

data_sample_submission = pd.read_csv(path + "sample_submission.csv")

<IPython.core.display.Javascript object>

In [4]:
data_col_desc

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...
217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


<IPython.core.display.Javascript object>

## <a name="C1.2"> Settings

In [5]:
# Set the results display
pd.set_option("display.max_rows", 70)
pd.set_option("display.max_columns", 100)
pd.options.display.float_format = "{:,.4f}".format

<IPython.core.display.Javascript object>

In [6]:
# General settings
random_set = 42
n_sample = 10000
test_size = 0.40
max_empty = 0.25

<IPython.core.display.Javascript object>

# <a name="C2"> Functions

In [7]:
# Function to remove features that has not enough values and replace empty values with more or median
def clean_df(df):
    col_to_ignore = df.columns[df.isnull().mean() > max_empty].to_list()
    df = df.drop(col_to_ignore, axis=1)
    col_to_fill = df.columns[df.isnull().mean() > 0].to_list()
    for col in col_to_fill:
        if df[col].dtypes == object:
            df.loc[df[col].isnull(), col] = df[col].mode()[0]
        else:
            df.loc[df[col].isnull(), col] = df[col].median()
    return df

<IPython.core.display.Javascript object>

In [8]:
# Fonction de traitement des outliers en fonction des quartiles
def outliers(df, var):
    # calculer les quartiles
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQ = Q3 - Q1
    mini = Q1 - 1.5 * IQ
    maxi = Q3 + 1.5 * IQ
    # mettre les outliers à NaN
    mask_mini = df[var] < mini
    df.loc[mask_mini, var] = np.NaN
    mask_maxi = df[var] > maxi
    df.loc[mask_maxi, var] = np.NaN

<IPython.core.display.Javascript object>

# <a name="C3"> Data analysis

In [9]:
data_col_desc["Table"].unique()

array(['application_{train|test}.csv', 'bureau.csv', 'bureau_balance.csv',
       'POS_CASH_balance.csv', 'credit_card_balance.csv',
       'previous_application.csv', 'installments_payments.csv'],
      dtype=object)

<IPython.core.display.Javascript object>

In [10]:
data_col_desc[data_col_desc["Table"] == "application_{train|test}.csv"]

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...
120,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN
121,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN
122,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN
123,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,NaN


<IPython.core.display.Javascript object>

In [11]:
data_sample_submission.describe()

,SK_ID_CURR,TARGET
count,"48,744.0000","48,744.0000"
mean,"277,796.6763",0.5000
std,"103,169.5473",0.0000
min,"100,001.0000",0.5000
25%,"188,557.7500",0.5000
50%,"277,549.0000",0.5000
75%,"367,555.5000",0.5000
max,"456,250.0000",0.5000


<IPython.core.display.Javascript object>

In [12]:
data_application_test.shape

(48744, 121)

<IPython.core.display.Javascript object>

In [13]:
data_application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,"202,500.0000","406,597.5000","24,700.5000","351,000.0000",Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.0188,-9461,-637,"-3,648.0000",-2120,NaN,1,1,0,1,1,0,Laborers,1.0000,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.0830,0.2629,0.1394,0.0247,0.0369,0.9722,0.6192,0.0143,0.0000,...,0.0250,0.0369,0.9722,0.6243,0.0144,0.0000,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0000,2.0000,2.0000,2.0000,"-1,134.0000",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
1,100003,0,Cash loans,F,N,N,0,"270,000.0000","1,293,502.5000","35,698.5000","1,129,500.0000",Family,State servant,Higher education,Married,House / apartment,0.0035,-16765,-1188,"-1,186.0000",-291,NaN,1,1,0,1,1,0,Core staff,2.0000,1,1,MONDAY,11,0,0,0,0,0,0,School,0.3113,0.6222,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.0800,...,0.0968,0.0529,0.9851,0.7987,0.0608,0.0800,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0000,0.0000,1.0000,0.0000,-828.0000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,100004,0,Revolving loans,M,Y,Y,0,"67,500.0000","135,000.0000","6,750.0000","135,000.0000",Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.0100,-19046,-225,"-4,260.0000",-2531,26.0000,1,1,1,1,1,0,Laborers,1.0000,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.5559,0.7296,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,-815.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,100006,0,Cash loans,F,N,Y,0,"135,000.0000","312,682.5000","29,686.5000","297,000.0000",Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.0080,-19005,-3039,"-9,833.0000",-2437,NaN,1,1,0,1,0,0,Laborers,2.0000,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,NaN,0.6504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

<IPython.core.display.Javascript object>

In [14]:
len(data_application_train) + len(data_application_test)

356255

<IPython.core.display.Javascript object>

In [15]:
data_application_train.describe(include="all")

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,"307,511.0000","307,511.0000",307511,307511,307511,307511,"307,511.0000","307,511.0000","307,511.0000","307,499.0000","307,233.0000",306219,307511,307511,307511,307511,"307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","104,582.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000",211120,"307,509.0000","307,511.0000","307,511.0000",307511,"307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000",307511,"134,133.0000","306,851.0000","246,546.0000","151,450.0000","127,568.0000","157,504.0000","103,023.0000","92,646.0000","143,620.0000",...,"151,450.0000","127,568.0000","157,504.0000","103,023.0000","92,646.0000","143,620.0000","152,683.0000","154,491.0000","98,869.0000","124,921.0000","97,312.0000","153,161.0000","93,997.0000","137,829.0000",97216,153214,"159,080.0000",151170,161756,"306,490.0000","306,490.0000","306,490.0000","306,490.0000","307,510.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","307,511.0000","265,992.0000","265,992.0000","265,992.0000","265,992.0000","265,992.0000","265,992.0000"
unique,NaN,NaN,2,3,2,2,NaN,NaN,NaN,NaN,NaN,7,8,5,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,3,NaN,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Cash loans,F,N,Y,NaN,NaN,NaN,NaN,NaN,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laborers,NaN,NaN,NaN,TUESDAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Entity Type 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,reg oper account,block of flats,NaN,Panel,No,NaN,NaN,NaN,NaN,NaN,NaN,Na

<IPython.core.display.Javascript object>

Lier les df et agréger les données pour obtenir une ligne par client (application) => P5
- ne retenir que les variables où les données sont toujours renseignées (ou quasiment)

Entrainer des modèles de classification binaire
- en déduire les X variables les plus importantes => P4

Afficher ces X variables dans un dashboard pour expliquer la prédiction du score
- pour un client sélectionné
- en comparant par rapport aux autres clients


In [16]:
print(data_previous_application.DAYS_FIRST_DRAWING.isnull().mean())
print(data_previous_application.DAYS_FIRST_DUE.isnull().mean())
print(data_previous_application.DAYS_LAST_DUE_1ST_VERSION.isnull().mean())
print(data_previous_application.DAYS_LAST_DUE.isnull().mean())
print(data_previous_application.DAYS_TERMINATION.isnull().mean())
print(data_previous_application.AMT_DOWN_PAYMENT.isnull().mean())
print(data_previous_application.RATE_DOWN_PAYMENT.isnull().mean())

0.40298129461254667
0.40298129461254667
0.40298129461254667
0.40298129461254667
0.40298129461254667
0.536364801157217
0.536364801157217


<IPython.core.display.Javascript object>

In [17]:
print(data_bureau.AMT_CREDIT_MAX_OVERDUE.isnull().mean())
print(data_bureau.AMT_CREDIT_SUM_LIMIT.isnull().mean())
print(data_bureau.AMT_ANNUITY.isnull().mean())
data_bureau[~data_bureau.AMT_ANNUITY.isnull()]

0.6551326359159837
0.3447741472406649
0.714734902949614


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
768,380361,5715448,Active,currency 1,-820,0,"31,069.0000",NaN,NaN,0,"67,500.0000",0.0000,"67,500.0000",0.0000,Credit card,-183,0.0000
769,380361,5715449,Active,currency 1,-357,0,"1,119.0000",NaN,NaN,0,"45,000.0000",0.0000,"45,000.0000",0.0000,Credit card,-130,"2,691.0000"
770,380361,5715451,Closed,currency 1,-917,0,-187.0000,-759.0000,NaN,0,"74,439.0000",0.0000,0.0000,0.0000,Consumer credit,-748,0.0000
771,380361,5715452,Closed,currency 1,-993,0,"31,039.0000",-831.0000,NaN,0,"315,000.0000",0.0000,0.0000,0.0000,Credit card,-818,0.0000
772,380361,5715453,Closed,currency 1,-1146,0,681.0000,-780.0000,NaN,0,"2,025,000.0000",0.0000,0.0000,0.0000,Consumer credit,-769,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716285,435426,5053678,Closed,currency 1,-1147,0,-416.0000,-849.0000,0.0000,0,"205,488.0000",0.0000,0.0000,0.0000,Consumer credit,-835,"94,378.5000"
1716286,377669,5053682,Closed,currency 1,-2215,0,"-1,843.0000","-1,653.0000",NaN,0,"58,027.5000",0.0000,0.0000,0.0000,Consumer credit,-1642,0.0000
1716287,431028,5053695,Closed,currency 1,-586,0,-191.0000,-177.0000,NaN,0,"1,350,000.0000",0.0000,NaN,0.0000,Consumer credit,-176,0.0000
1716290,363575,5053740,Closed,currency 1,-1234,0,-503.0000,-502.0000,NaN,0,"900,000.0000",0.0000,0.0000,0.0000,Consumer credit,-461,"58,554.0000"


<IPython.core.display.Javascript object>

In [18]:
data_application_test = clean_df(data_application_test)
print("data_application_test :", data_application_test.shape)

data_application_train = clean_df(data_application_train)
print("data_application_train :", data_application_train.shape)

data_bureau = clean_df(data_bureau)
print("data_bureau :", data_bureau.shape)

data_bureau_balance = clean_df(data_bureau_balance)
print("data_bureau_balance :", data_bureau_balance.shape)

data_credit_card_balance = clean_df(data_credit_card_balance)
print("data_credit_card_balance :", data_credit_card_balance.shape)

data_installments_payments = clean_df(data_installments_payments)
print("data_installments_payments :", data_installments_payments.shape)

data_POS_CASH_balance = clean_df(data_POS_CASH_balance)
print("data_POS_CASH_balance :", data_POS_CASH_balance.shape)

data_previous_application = clean_df(data_previous_application)
print("data_previous_application :", data_previous_application.shape)

data_application_test : (48744, 71)
data_application_train : (307511, 72)
data_bureau : (1716428, 13)
data_bureau_balance : (27299925, 3)
data_credit_card_balance : (3840312, 23)
data_installments_payments : (13605401, 8)
data_POS_CASH_balance : (10001358, 8)
data_previous_application : (1670214, 26)


<IPython.core.display.Javascript object>

In [19]:
data_previous_application.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
       'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
print(
    "data_application_test :",
    len(data_application_test.columns[data_application_test.isnull().mean() > 0])
    / len(data_application_test),
)

print(
    "data_application_train :",
    len(data_application_train.columns[data_application_train.isnull().mean() > 0])
    / len(data_application_train),
)

print(
    "data_bureau :",
    len(data_bureau.columns[data_bureau.isnull().mean() > 0]) / len(data_bureau),
)

print(
    "data_bureau_balance :",
    len(data_bureau_balance.columns[data_bureau_balance.isnull().mean() > 0])
    / len(data_bureau_balance),
)

print(
    "data_credit_card_balance :",
    len(data_credit_card_balance.columns[data_credit_card_balance.isnull().mean() > 0])
    / len(data_credit_card_balance),
)

print(
    "data_installments_payments :",
    len(
        data_installments_payments.columns[
            data_installments_payments.isnull().mean() > 0
        ]
    )
    / len(data_installments_payments),
)

print(
    "data_POS_CASH_balance :",
    len(data_POS_CASH_balance.columns[data_POS_CASH_balance.isnull().mean() > 0])
    / len(data_POS_CASH_balance),
)

print(
    "data_previous_application :",
    len(
        data_previous_application.columns[data_previous_application.isnull().mean() > 0]
    )
    / len(data_previous_application),
)

data_application_test : 0.0
data_application_train : 0.0
data_bureau : 0.0
data_bureau_balance : 0.0
data_credit_card_balance : 0.0
data_installments_payments : 0.0
data_POS_CASH_balance : 0.0
data_previous_application : 0.0


<IPython.core.display.Javascript object>

In [21]:
# %% [code]
# HOME CREDIT DEFAULT RISK COMPETITION
# Most features are created by applying min, max, mean, sum and var functions to grouped tables.
# Little feature selection is done and overfitting might be a problem since many features are related.
# The following key ideas were used:
# - Divide or subtract important features to get rates (like annuity and income)
# - In Bureau Data: create specific features for Active credits and Closed credits
# - In Previous Applications: create specific features for Approved and Refused applications
# - Modularity: one function for each table (except bureau_balance and application_test)
# - One-hot encoding for categorical features
# All tables are joined with the application DF using the SK_ID_CURR key (except bureau_balance).
# You can use LightGBM with KFold or Stratified KFold.

# Update 16/06/2018:
# - Added Payment Rate feature
# - Removed index from features
# - Use standard KFold CV (not stratified)

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager

# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=False):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == "object"]
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows=None, nan_as_category=False):
    # Read data and merge
    df = data_application_train
    # test_df = data_application_test
    # print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    # df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df["CODE_GENDER"] != "XNA"]

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ["CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY"]:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)

    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)
    # Some simple new features (percentages)
    df["DAYS_EMPLOYED_PERC"] = df["DAYS_EMPLOYED"] / df["DAYS_BIRTH"]
    df["INCOME_CREDIT_PERC"] = df["AMT_INCOME_TOTAL"] / df["AMT_CREDIT"]
    df["INCOME_PER_PERSON"] = df["AMT_INCOME_TOTAL"] / df["CNT_FAM_MEMBERS"]
    df["ANNUITY_INCOME_PERC"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]
    df["PAYMENT_RATE"] = df["AMT_ANNUITY"] / df["AMT_CREDIT"]
    # del test_df
    gc.collect()
    return df


# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=False):
    bureau = data_bureau
    bb = data_bureau_balance
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {"MONTHS_BALANCE": ["min", "max", "size"]}
    for col in bb_cat:
        bb_aggregations[col] = ["mean"]
    bb_agg = bb.groupby("SK_ID_BUREAU").agg(bb_aggregations)
    bb_agg.columns = pd.Index(
        [e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()]
    )
    bureau = bureau.join(bb_agg, how="left", on="SK_ID_BUREAU")
    bureau.drop(["SK_ID_BUREAU"], axis=1, inplace=True)
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        "DAYS_CREDIT": ["min", "max", "mean", "var"],
        "DAYS_CREDIT_ENDDATE": ["min", "max", "mean"],
        "DAYS_CREDIT_UPDATE": ["mean"],
        "CREDIT_DAY_OVERDUE": ["max", "mean"],
        # "AMT_CREDIT_MAX_OVERDUE": ["mean"],
        "AMT_CREDIT_SUM": ["max", "mean", "sum"],
        "AMT_CREDIT_SUM_DEBT": ["max", "mean", "sum"],
        "AMT_CREDIT_SUM_OVERDUE": ["mean"],
        # "AMT_CREDIT_SUM_LIMIT": ["mean", "sum"],
        # "AMT_ANNUITY": ["max", "mean"],
        "CNT_CREDIT_PROLONG": ["sum"],
        "MONTHS_BALANCE_MIN": ["min"],
        "MONTHS_BALANCE_MAX": ["max"],
        "MONTHS_BALANCE_SIZE": ["mean", "sum"],
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat:
        cat_aggregations[cat] = ["mean"]
    for cat in bb_cat:
        cat_aggregations[cat + "_MEAN"] = ["mean"]

    bureau_agg = bureau.groupby("SK_ID_CURR").agg(
        {**num_aggregations, **cat_aggregations}
    )
    bureau_agg.columns = pd.Index(
        ["BURO_" + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()]
    )
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau["CREDIT_ACTIVE_Active"] == 1]
    active_agg = active.groupby("SK_ID_CURR").agg(num_aggregations)
    active_agg.columns = pd.Index(
        ["ACTIVE_" + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()]
    )
    bureau_agg = bureau_agg.join(active_agg, how="left", on="SK_ID_CURR")
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau["CREDIT_ACTIVE_Closed"] == 1]
    closed_agg = closed.groupby("SK_ID_CURR").agg(num_aggregations)
    closed_agg.columns = pd.Index(
        ["CLOSED_" + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()]
    )
    bureau_agg = bureau_agg.join(closed_agg, how="left", on="SK_ID_CURR")
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg


# Preprocess previous_applications.csv
def previous_applications(num_rows=None, nan_as_category=False):
    prev = data_previous_application
    prev, cat_cols = one_hot_encoder(prev, nan_as_category)
    # Days 365.243 values -> nan
    # prev["DAYS_FIRST_DRAWING"].replace(365243, np.nan, inplace=True)
    # prev["DAYS_FIRST_DUE"].replace(365243, np.nan, inplace=True)
    # prev["DAYS_LAST_DUE_1ST_VERSION"].replace(365243, np.nan, inplace=True)
    # prev["DAYS_LAST_DUE"].replace(365243, np.nan, inplace=True)
    # prev["DAYS_TERMINATION"].replace(365243, np.nan, inplace=True)
    # Add feature: value ask / value received percentage
    prev["APP_CREDIT_PERC"] = prev["AMT_APPLICATION"] / prev["AMT_CREDIT"]
    # Previous applications numeric features
    num_aggregations = {
        "AMT_ANNUITY": ["min", "max", "mean"],
        "AMT_APPLICATION": ["min", "max", "mean"],
        "AMT_CREDIT": ["min", "max", "mean"],
        "APP_CREDIT_PERC": ["min", "max", "mean", "var"],
        # "AMT_DOWN_PAYMENT": ["min", "max", "mean"],
        "AMT_GOODS_PRICE": ["min", "max", "mean"],
        "HOUR_APPR_PROCESS_START": ["min", "max", "mean"],
        # "RATE_DOWN_PAYMENT": ["min", "max", "mean"],
        "DAYS_DECISION": ["min", "max", "mean"],
        "CNT_PAYMENT": ["mean", "sum"],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ["mean"]

    prev_agg = prev.groupby("SK_ID_CURR").agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(
        ["PREV_" + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()]
    )
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev["NAME_CONTRACT_STATUS_Approved"] == 1]
    approved_agg = approved.groupby("SK_ID_CURR").agg(num_aggregations)
    approved_agg.columns = pd.Index(
        ["APPROVED_" + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()]
    )
    prev_agg = prev_agg.join(approved_agg, how="left", on="SK_ID_CURR")
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev["NAME_CONTRACT_STATUS_Refused"] == 1]
    refused_agg = refused.groupby("SK_ID_CURR").agg(num_aggregations)
    refused_agg.columns = pd.Index(
        ["REFUSED_" + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()]
    )
    prev_agg = prev_agg.join(refused_agg, how="left", on="SK_ID_CURR")
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg


# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows=None, nan_as_category=False):
    pos = data_POS_CASH_balance
    pos, cat_cols = one_hot_encoder(pos, nan_as_category)
    # Features
    aggregations = {
        "MONTHS_BALANCE": ["max", "mean", "size"],
        "SK_DPD": ["max", "mean"],
        "SK_DPD_DEF": ["max", "mean"],
    }
    for cat in cat_cols:
        aggregations[cat] = ["mean"]

    pos_agg = pos.groupby("SK_ID_CURR").agg(aggregations)
    pos_agg.columns = pd.Index(
        ["POS_" + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()]
    )
    # Count pos cash accounts
    pos_agg["POS_COUNT"] = pos.groupby("SK_ID_CURR").size()
    del pos
    gc.collect()
    return pos_agg


# Preprocess installments_payments.csv
def installments_payments(num_rows=None, nan_as_category=False):
    ins = data_installments_payments
    ins, cat_cols = one_hot_encoder(ins, nan_as_category)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins["PAYMENT_PERC"] = ins["AMT_PAYMENT"] / ins["AMT_INSTALMENT"]
    ins["PAYMENT_DIFF"] = ins["AMT_INSTALMENT"] - ins["AMT_PAYMENT"]
    # Days past due and days before due (no negative values)
    ins["DPD"] = ins["DAYS_ENTRY_PAYMENT"] - ins["DAYS_INSTALMENT"]
    ins["DBD"] = ins["DAYS_INSTALMENT"] - ins["DAYS_ENTRY_PAYMENT"]
    ins["DPD"] = ins["DPD"].apply(lambda x: x if x > 0 else 0)
    ins["DBD"] = ins["DBD"].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        "NUM_INSTALMENT_VERSION": ["nunique"],
        "DPD": ["max", "mean", "sum"],
        "DBD": ["max", "mean", "sum"],
        "PAYMENT_PERC": ["max", "mean", "sum", "var"],
        "PAYMENT_DIFF": ["max", "mean", "sum", "var"],
        "AMT_INSTALMENT": ["max", "mean", "sum"],
        "AMT_PAYMENT": ["min", "max", "mean", "sum"],
        "DAYS_ENTRY_PAYMENT": ["max", "mean", "sum"],
    }
    for cat in cat_cols:
        aggregations[cat] = ["mean"]
    ins_agg = ins.groupby("SK_ID_CURR").agg(aggregations)
    ins_agg.columns = pd.Index(
        ["INSTAL_" + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()]
    )
    # Count installments accounts
    ins_agg["INSTAL_COUNT"] = ins.groupby("SK_ID_CURR").size()
    del ins
    gc.collect()
    return ins_agg


# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows=None, nan_as_category=False):
    cc = data_credit_card_balance
    cc, cat_cols = one_hot_encoder(cc, nan_as_category)
    # General aggregations
    cc.drop(["SK_ID_PREV"], axis=1, inplace=True)
    cc_agg = cc.groupby("SK_ID_CURR").agg(["min", "max", "mean", "sum", "var"])
    cc_agg.columns = pd.Index(
        ["CC_" + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()]
    )
    # Count credit card lines
    cc_agg["CC_COUNT"] = cc.groupby("SK_ID_CURR").size()
    del cc
    gc.collect()
    return cc_agg

<IPython.core.display.Javascript object>

In [22]:
df = application_train_test()
print(df.shape)

bureau_agg = bureau_and_balance()
print(bureau_agg.shape)

prev_agg = previous_applications()
print(prev_agg.shape)

pos_agg = pos_cash()
print(pos_agg.shape)

ins_agg = installments_payments()
print(ins_agg.shape)

cc_agg = credit_card_balance()
print(cc_agg.shape)

C:\Users\bruno\AppData\Local\Temp\ipykernel_8524\3516951796.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[bin_feature], uniques = pd.factorize(df[bin_feature])


(307507, 168)
(305811, 97)
(338857, 208)
(337252, 17)
(339587, 26)
(103558, 136)


<IPython.core.display.Javascript object>

In [23]:
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,...,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100002,1,0,0,0,0,"202,500.0000","406,597.5000","24,700.5000","351,000.0000",0.0188,-9461,-637.0000,"-3,648.0000",-2120,1,1,0,1,1,0,1.0000,2,2,10,0,0,0,0,0,0,0.2629,0.1394,2.0000,2.0000,2.0000,2.0000,"-1,134.0000",0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0673,0.4980,"202,500.0000",0.1220,0.0607
1,100003,0,1,0,1,0,"270,000.0000","1,293,502.5000","35,698.5000","1,129,500.0000",0.0035,-16765,"-1,188.0000","-1,186.0000",-291,1,1,0,1,1,0,2.0000,1,1,11,0,0,0,0,0,0,0.6222,0.5353,1.0000,0.0000,1.0000,0.0000,-828.0000,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0709,0.2087,"135,000.0000",0.1322,0.0276
2,100004,0,0,1,0,0,"67,500.0000","135,000.0000","6,750.0000","135,000.0000",0.0100,-19046,-225.0000,"-4,260.0000",-2531,1,1,1,1,1,0,1.0000,2,2,9,0,0,0,0,0,0,0.5559,0.7296,0.0000,0.0000,0.0000,0.0000,-815.0000,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0118,0.5000,"67,500.0000",0.1000,0.0500
3,100006,0,1,0,0,0,"135,000.0000","312,682.5000","29,686.5000","297,000.0000",0.0080,-19005,"-3,039.0000","-9,833.0000",-2437,1,1,0,1,0,0,2.0000,2,2,17,0,0,0,0,0,0,0.6504,0.5353,2.0000,0.0000,2.0000,0.0000,-617.0000,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1599,0.4317,"67,500.0000",0.2199,0.0949
4,100007,0,0,0,0,0,"121,500.0000","513,000.0000","21,865.5000","513,000.0000",0.0287,-19932,"-3,038.0000","-4,311.0000

<IPython.core.display.Javascript object>

In [24]:
data_merged = pd.merge(df, bureau_agg, on="SK_ID_CURR", how="left")
data_merged = pd.merge(data_merged, prev_agg, on="SK_ID_CURR", how="left")
data_merged = pd.merge(data_merged, pos_agg, on="SK_ID_CURR", how="left")
data_merged = pd.merge(data_merged, ins_agg, on="SK_ID_CURR", how="left")
data_merged = pd.merge(data_merged, cc_agg, on="SK_ID_CURR", how="left")

<IPython.core.display.Javascript object>

In [25]:
data_merged.shape

(307507, 652)

<IPython.core.display.Javascript object>

In [26]:
empty_cols = data_merged.columns[data_merged.isnull().mean() > 0].to_list()
len(empty_cols) / len(data_merged.columns)

0.745398773006135

<IPython.core.display.Javascript object>

In [27]:
empty_cols

['DAYS_EMPLOYED',
 'DAYS_EMPLOYED_PERC',
 'BURO_DAYS_CREDIT_MIN',
 'BURO_DAYS_CREDIT_MAX',
 'BURO_DAYS_CREDIT_MEAN',
 'BURO_DAYS_CREDIT_VAR',
 'BURO_DAYS_CREDIT_ENDDATE_MIN',
 'BURO_DAYS_CREDIT_ENDDATE_MAX',
 'BURO_DAYS_CREDIT_ENDDATE_MEAN',
 'BURO_DAYS_CREDIT_UPDATE_MEAN',
 'BURO_CREDIT_DAY_OVERDUE_MAX',
 'BURO_CREDIT_DAY_OVERDUE_MEAN',
 'BURO_AMT_CREDIT_SUM_MAX',
 'BURO_AMT_CREDIT_SUM_MEAN',
 'BURO_AMT_CREDIT_SUM_SUM',
 'BURO_AMT_CREDIT_SUM_DEBT_MAX',
 'BURO_AMT_CREDIT_SUM_DEBT_MEAN',
 'BURO_AMT_CREDIT_SUM_DEBT_SUM',
 'BURO_AMT_CREDIT_SUM_OVERDUE_MEAN',
 'BURO_CNT_CREDIT_PROLONG_SUM',
 'BURO_MONTHS_BALANCE_MIN_MIN',
 'BURO_MONTHS_BALANCE_MAX_MAX',
 'BURO_MONTHS_BALANCE_SIZE_MEAN',
 'BURO_MONTHS_BALANCE_SIZE_SUM',
 'BURO_CREDIT_ACTIVE_Active_MEAN',
 'BURO_CREDIT_ACTIVE_Bad debt_MEAN',
 'BURO_CREDIT_ACTIVE_Closed_MEAN',
 'BURO_CREDIT_ACTIVE_Sold_MEAN',
 'BURO_CREDIT_CURRENCY_currency 1_MEAN',
 'BURO_CREDIT_CURRENCY_currency 2_MEAN',
 'BURO_CREDIT_CURRENCY_currency 3_MEAN',
 'BURO_CREDI

<IPython.core.display.Javascript object>

In [28]:
features_analysis = list(data_merged.columns)
del features_analysis[0:2]
print(len(features_analysis))

col_one = []
col_std = []

for col in features_analysis:
    if data_merged[col].min() == 0 and data_merged[col].max() == 1:
        col_one.append(col)
    else:
        col_std.append(col)

print(len(col_one))
print(len(col_std))
features_analysis[0:10]

650
307
343


['CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH']

<IPython.core.display.Javascript object>

In [29]:
data_merged.isnull().mean().value_counts(bins=4)

(-0.00184, 0.21]    434
(0.63, 0.84]        177
(0.21, 0.42]         39
(0.42, 0.63]          2
dtype: int64

<IPython.core.display.Javascript object>

In [30]:
# Manage outliers
for col in col_std:
    outliers(data_merged, col)

<IPython.core.display.Javascript object>

In [31]:
data_merged.isnull().mean().value_counts(bins=4)

(-0.001863, 0.216]    417
(0.648, 0.863]        177
(0.216, 0.432]         55
(0.432, 0.648]          3
dtype: int64

<IPython.core.display.Javascript object>

In [32]:
col_to_delete = data_merged.columns[data_merged.isnull().mean() > max_empty].to_list()
data_merged = data_merged.drop(col_to_delete, axis=1)
data_merged.shape

(307507, 427)

<IPython.core.display.Javascript object>

In [33]:
data_transp = data_merged.transpose()
data_transp.isnull().mean().value_counts(bins=4)

(-0.001632, 0.158]    290100
(0.474, 0.632]         15523
(0.158, 0.316]           953
(0.316, 0.474]           931
dtype: int64

<IPython.core.display.Javascript object>

In [34]:
index_to_delete = data_transp.columns[data_transp.isnull().mean() > max_empty].to_list()
data_merged = data_merged.drop(index_to_delete, axis=0)
data_merged.shape

(291034, 427)

<IPython.core.display.Javascript object>

In [35]:
data_merged.isnull().mean().value_counts(bins=4)

(-0.00124, 0.0601]    328
(0.12, 0.18]           44
(0.0601, 0.12]         43
(0.18, 0.24]           12
dtype: int64

<IPython.core.display.Javascript object>

In [36]:
data_final = clean_df(data_merged)
data_final.shape

(291034, 427)

<IPython.core.display.Javascript object>

In [37]:
data_final.isnull().mean().value_counts(bins=4)

(-0.0005, 0.0]       427
(-0.002, -0.0005]      0
(0.0, 0.0005]          0
(0.0005, 0.001]        0
dtype: int64

<IPython.core.display.Javascript object>

In [38]:
np.isinf(data_final).mean().value_counts(bins=4)

(-0.0005, 0.0]       427
(-0.002, -0.0005]      0
(0.0, 0.0005]          0
(0.0005, 0.001]        0
dtype: int64

<IPython.core.display.Javascript object>

In [39]:
data_final.describe()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,...,APPROVED_AMT_GOODS_PRICE_MAX,APPROVED_AMT_GOODS_PRICE_MEAN,APPROVED_HOUR_APPR_PROCESS_START_MIN,APPROVED_HOUR_APPR_PROCESS_START_MAX,APPROVED_HOUR_APPR_PROCESS_START_MEAN,APPROVED_DAYS_DECISION_MIN,APPROVED_DAYS_DECISION_MAX,APPROVED_DAYS_DECISION_MEAN,APPROVED_CNT_PAYMENT_MEAN,APPROVED_CNT_PAYMENT_SUM,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_COUNT,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_SUM,INSTAL_PAYMENT_PERC_VAR,INSTAL_PAYMENT_DIFF_MAX,INSTAL_PAYMENT_DIFF_VAR,INSTAL_AMT_INSTALMENT_MAX,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
count,"291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000",...,"291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000","291,034.0000"
mean,"278,218.1601",0.0819,0.6623,0.3384,0.2932,0.3773,"153,220.4874","565,113.4079","25,932.1423","483,471.3993",0.0193,"-16,050.8476","-1,864.2165","-4,968.7110","-2,992.2757",1.0000,0.8193,0.1993,0.9993,0.2824,0.0582,2.1176,12.0527,0.0142,0.0492,0.0399,0.0781,0.2316,0.1809,0

<IPython.core.display.Javascript object>

In [40]:
features_analysis = list(data_final.columns)
del features_analysis[0:2]

col_one = []
col_std = []

for col in features_analysis:
    if data_final[col].min() == 0 and data_final[col].max() == 1:
        col_one.append(col)
    else:
        col_std.append(col)

features_analysis = col_std + col_one

print(len(features_analysis))
print(len(col_one))
print(len(col_std))
print(features_analysis[0:10])

425
283
142
['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']


<IPython.core.display.Javascript object>

In [41]:
# Paramètres
n_alphas = 10
Cs = np.logspace(-5, 5, n_alphas)
tolerance = [0.01]
kernel = ["rbf"]

splits = 10

score = metrics.make_scorer(metrics.fbeta_score, beta=2)
score_name = "f beta score"

alphas = np.logspace(-5, 5, n_alphas)
gammas = np.logspace(-5, 5, n_alphas)
layers = range(20, 30, 5)
iteration = [4000]
estimators = range(500, 700, 100)
l1 = np.arange(0.7, 1, 0.1)
learning_rate = np.arange(0.015, 0.030, 0.005)
iter_calc = [12000]

# Initiate models
svc = svm.SVC(cache_size=6000, probability=True, class_weight="balanced")
krc = kernel_ridge.KernelRidge()
rfc = RandomForestClassifier(oob_score=True, random_state=random_set)
gbc = GradientBoostingClassifier(random_state=random_set)
mlp = MLPClassifier(random_state=random_set)
# light gbm

<IPython.core.display.Javascript object>

In [42]:
data_final["TARGET"].value_counts()
# respecter la proportion de 0 et 1 dans le sample

0    267190
1     23844
Name: TARGET, dtype: int64

<IPython.core.display.Javascript object>

In [43]:
prop_0 = round(data_final["TARGET"].value_counts()[0] / len(data_final), 2)
prop_1 = round(1 - prop_0, 2)
print(prop_0)
print(prop_1)
n_sample_0 = round(n_sample * prop_0)
n_sample_1 = n_sample - n_sample_0
print(n_sample_0)
print(n_sample_1)

0.92
0.08
9200
800


<IPython.core.display.Javascript object>

In [44]:
# Features selection
k_features = 25
y = data_final["TARGET"]
X = data_final[features_analysis]

# Only scale relevant columns
# StandardScaler
# RobustScaler
# MinMaxScaler
scale = ColumnTransformer(
    transformers=[
        ("std", StandardScaler(), col_std),
        ("no_std", "passthrough", col_one),
    ]
)

scale.fit(X)
X_std = scale.transform(X)
print(X_std.shape)

# Let's keep the 25 most important features
feature_selection = SelectKBest(k=k_features).fit(X_std, y)
new_feat = X.columns[feature_selection.get_support()]
new_feat

(291034, 425)


C:\Users\bruno\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 10  16  18  20  21  22  23  24  26  27  39  40  47  48  49  50  51  52
  53  54  78  79  80  81  82  83 110 111 112 113 114 115 116 117 126] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\bruno\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Index(['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_EMPLOYED_PERC',
       'BURO_DAYS_CREDIT_MIN', 'BURO_DAYS_CREDIT_MAX', 'BURO_DAYS_CREDIT_MEAN',
       'BURO_DAYS_CREDIT_ENDDATE_MIN', 'BURO_DAYS_CREDIT_UPDATE_MEAN',
       'PREV_DAYS_DECISION_MIN', 'APPROVED_DAYS_DECISION_MIN', 'CODE_GENDER',
       'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Higher education',
       'BURO_CREDIT_ACTIVE_Active_MEAN', 'BURO_CREDIT_ACTIVE_Closed_MEAN',
       'PREV_NAME_CONTRACT_STATUS_Approved_MEAN',
       'PREV_NAME_CONTRACT_STATUS_Refused_MEAN',
       'PREV_CODE_REJECT_REASON_HC_MEAN', 'PREV_CODE_REJECT_REASON_SCOFR_MEAN',
       'PREV_CODE_REJECT_REASON_XAP_MEAN',
       'PREV_NAME_PRODUCT_TYPE_walk-in_MEAN'],
      dtype='object')

<IPython.core.display.Javascript object>

In [45]:
col_one = []
col_std = []

for col in new_feat:
    if data_final[col].min() == 0 and data_final[col].max() == 1:
        col_one.append(col)
    else:
        col_std.append(col)

features_analysis = col_std + col_one

print(len(features_analysis))
print(len(col_one))
print(len(col_std))
print(features_analysis[0:10])

25
11
14
['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_EMPLOYED_PERC', 'BURO_DAYS_CREDIT_MIN', 'BURO_DAYS_CREDIT_MAX', 'BURO_DAYS_CREDIT_MEAN']


<IPython.core.display.Javascript object>

In [46]:
# Data sample
data_0 = data_final[data_final["TARGET"] == 0].sample(
    n_sample_0, random_state=random_set
)
data_1 = data_final[data_final["TARGET"] == 1].sample(
    n_sample_1, random_state=random_set
)
data = pd.concat([data_0, data_1])

y = data["TARGET"]
X = data[features_analysis]

# train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

print("Training data :", X_train.shape)
print("Testing data :", X_test.shape)


# Scalers
scale_std = ColumnTransformer(
    transformers=[
        ("std", StandardScaler(), col_std),
        ("no_std", "passthrough", col_one),
    ]
)

scale_robust = ColumnTransformer(
    transformers=[
        ("std", RobustScaler(), col_std),
        ("no_std", "passthrough", col_one),
    ]
)

scale_min_max = ColumnTransformer(
    transformers=[
        ("std", MinMaxScaler(), col_std),
        ("no_std", "passthrough", col_one),
    ]
)

scale_std.fit(X_train)
X_test_std = scale_std.transform(X_test)

# Oversampling
smote = SMOTE(sampling_strategy=0.8, random_state=random_set)

# fit predictor and target variable
X_train, y_train = smote.fit_resample(X_train, y_train)
print("SMOTEd training data :", X_train.shape)

X_train_std = scale_std.transform(X_train)

# Classifiers
svc = svm.SVC(cache_size=6000, probability=True)
rfc = RandomForestClassifier(oob_score=True, random_state=random_set)
# gbc = GradientBoostingClassifier(random_state=random_set)
# mlp = MLPClassifier(random_state=random_set)
lgbm = LGBMClassifier()

# model_names = [svc, rfc, gbc, mlp, lgbm]
model_names = [svc, rfc, lgbm]

# SelectKBest
feature_selection = SelectKBest(k=k_features)

# pipeline_std = Pipeline(steps=[("scaler", scale_std), ("classifier", svc)])
pipeline_std = Pipeline(steps=[("classifier", svc)])
pipeline_robust = Pipeline(steps=[("scaler", scale_robust), ("classifier", svc)])
pipeline_min_max = Pipeline(steps=[("scaler", scale_min_max), ("classifier", svc)])

# SVC
params_svc = {}
params_svc["classifier__C"] = Cs
params_svc["classifier__tol"] = tolerance
params_svc["classifier__kernel"] = kernel
params_svc["classifier"] = [svc]

# Light GBM
params_lgbm = {}
params_lgbm["classifier__n_estimators"] = estimators
params_lgbm["classifier__learning_rate"] = learning_rate
params_lgbm["classifier"] = [lgbm]

# Random Forest Classifier
params_rfc = {}
params_rfc["classifier__n_estimators"] = estimators
params_rfc["classifier"] = [rfc]

params = [params_svc, params_lgbm, params_rfc]

gs_std = GridSearchCV(pipeline_std, params, scoring=score, cv=splits)
gs_robust = GridSearchCV(pipeline_robust, params, scoring=score, cv=splits)
gs_min_max = GridSearchCV(pipeline_min_max, params, scoring=score, cv=splits)

Training data : (6000, 25)
Testing data : (4000, 25)
SMOTEd training data : (9887, 25)


<IPython.core.display.Javascript object>

In [ ]:
gs_std.fit(X_train_std, y_train)
print("Best params :", gs_std.best_params_)
print(score_name, " :", gs_std.best_score_)

In [ ]:
X_test_std

In [ ]:
# Calculate predictions and probabilities
y_pred = gs_std.predict(X_test_std)
y_prob = gs_std.predict_proba(X_test_std)[:, 1]

print("F1 score :", metrics.f1_score(y_test, y_pred))
print("F beta score :", metrics.fbeta_score(y_test, y_pred, beta=2))
print("Precision :", metrics.precision_score(y_test, y_pred))
print("Recall :", metrics.recall_score(y_test, y_pred))
print("ROC score :", metrics.roc_auc_score(y_test, y_prob))
print("Confusion matrix : ")
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
[fpr, tpr, thr] = metrics.roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, color="coral", lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("1 - specificite", fontsize=14)
plt.ylabel("Sensibilite", fontsize=14)

In [ ]:
# Regardons pour notre meilleur modèle quelles sont les 10 variables les plus influentes
from sklearn.inspection import permutation_importance

r = permutation_importance(
    gs_std, X_train, y_train, n_repeats=5, random_state=random_set
)
for i in r.importances_mean.argsort()[::-1][:10]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(
            f"{X_train.columns[i]:<8} : "
            f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}"
        )

In [ ]:
# SHAP (pour feature importance)